# Exercises

This is the notebook containing the exercises for Feature Store, Model Monitor, and Clarify. Tested for these exercises was performed using __2 vCPU + 4 GiB notebook instance with Python 3 (TensorFlow 2.1 Python 3.6 CPU Optimized) kernel__.

## Staging

We'll begin by initializing some variables. These are often assumed to be present in code samples you'll find in the AWS documenation.

In [47]:
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role

role = get_execution_role()
session = sagemaker.Session()
region = session.boto_region_name
bucket = session.default_bucket()

## Feature Store
---

Feature store is a special database to give ML systems a consistent data flow across training and inference workloads. It can ingest data in batches (for training) as well as serve input features to models with very low latency for real-time prediction.

For this exercise we'll work with a wine quality dataset: https://archive.ics.uci.edu/ml/datasets/wine+quality/

```P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.```

In [48]:
import pandas as pd
from sklearn import datasets
import time
import uuid

In [49]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']

If we leave the column names as-is, Feature Store won't be able to handle the `/` in `od280/od315_of_diluted_wines` (`/` is a delimiter Feature Store uses to manage how features are organized.)

In [50]:
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

Once we have our data, we can create a feature group. Remember to attach event time and ID columns - Feature Store needs them.

In [51]:
# Add a feature group
df["EventTime"] = time.time()
df["id"] = range(len(df))

# TODO: Create feature group
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group = FeatureGroup(
    name="wine-features", sagemaker_session=session
)


# TODO: Load Feature definitions
feature_group.load_feature_definitions(data_frame=df)

[FeatureDefinition(feature_name='alcohol', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='malic_acid', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='ash', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='alcalinity_of_ash', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='magnesium', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='total_phenols', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='flavanoids', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='nonflavanoid_phenols', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='proanthocyanins', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='color_intensity',

The feature group is not created until we call the `create` method, let's do that now:

In [52]:
# Create the feature store:

In [53]:
# TODO
feature_group.create(
    s3_uri=f"s3://{bucket}/features",
    record_identifier_name='id',
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True,
)

ResourceInUse: An error occurred (ResourceInUse) when calling the CreateFeatureGroup operation: Resource Already Exists: FeatureGroup with name wine-features already exists. Choose a different name.

Lastly, ingest some data into your feature group:

In [54]:
# TODO
feature_group.ingest(data_frame=df, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='wine-features', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fbb3dfc3160>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7fbb3de4cdd8>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

Great job! You've demonstrated your understanding of creating feature groups and ingesting data into them using Feature Store. Next up we'll cover Model Monitor!

## Model Monitor

In this exercise we'll create a monitoring schedule for a deployed model. We're going to provide code to help you deploy a model and get started, so that you can focus on Model Monitor for this exercise. __Remember to clean up your model before you end a work session__. We'll provide some code at the end to help you clean up your model. We'll begin by reloading our data from the previous exercise.



In [55]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

We also need to put the target variable in the first column per the docs for our chosen algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [56]:
df["TARGET"] = data['target']
df.set_index(df.pop('TARGET'), inplace=True)
df.reset_index(inplace=True)

Now we'll upload the data to S3 as train and validation data:

In [57]:
delimiter = int(len(df)/2)
train, test = df.iloc[delimiter:], df.iloc[:delimiter]

train.to_csv("train.csv", header=False, index=False)
test.to_csv("validation.csv", header=False, index=False)

val_location = session.upload_data('./validation.csv', key_prefix="data")
train_location = session.upload_data('./train.csv', key_prefix="data")

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

In [58]:
algo_image = sagemaker.image_uris.retrieve("xgboost", region, version='latest')
s3_output_location = f"s3://{bucket}/models/wine_model"

model=sagemaker.estimator.Estimator(
    image_uri=algo_image,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session()
)

model.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='reg:linear',
                        early_stopping_rounds=10,
                        num_round=200)


model.fit({'train': s3_input_train, 'validation': s3_input_validation})

2022-01-14 14:15:49 Starting - Starting the training job...
2022-01-14 14:16:13 Starting - Launching requested ML instancesProfilerReport-1642169749: InProgress
......
2022-01-14 14:17:13 Starting - Preparing the instances for training.........
2022-01-14 14:18:48 Downloading - Downloading input data...
2022-01-14 14:19:14 Training - Downloading the training image..Arguments: train
[2022-01-14:14:19:23:INFO] Running standalone xgboost training.
[2022-01-14:14:19:23:INFO] File size need to be processed in the node: 0.01mb. Available memory size in the node: 8343.7mb
[2022-01-14:14:19:23:INFO] Determined delimiter of CSV input is ','
[14:19:23] S3DistributionType set as FullyReplicated
[14:19:23] 89x13 matrix with 1157 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-01-14:14:19:23:INFO] Determined delimiter of CSV input is ','
[14:19:23] S3DistributionType set as FullyReplicated
[14:19:23] 89x13 matrix with 1157 entries loaded from /opt/ml/input/d

Now that your training job has finished, you can perform the first task in this exercise: creating a data capture config. Configure your model to sample `34%` of inferences:

In [69]:
# TODO
from sagemaker.model_monitor import DataCaptureConfig

capture_uri = f's3://{bucket}/data-capture'

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=capture_uri
)

Great! We'll use your config to deploy a model below:

In [70]:
xgb_predictor = model.deploy(
    initial_instance_count=1, instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)

-----------------!

Great! You should see an indicator like this when the deployment finishes:

```
-----------------!
```
We can test your deployment like so:

In [16]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()
inputs = test.copy()
# Drop the target variable
inputs = inputs.drop(columns=inputs.columns[0])
x_pred = xgb_predictor.predict(inputs.sample(5).values).decode('utf-8')
x_pred

'0.6030303239822388,0.6030303239822388,0.6030303239822388,0.6030303239822388,0.6030303239822388'

All systems go! To finish up the exercise, we're going to provide you with a DefaultModelMonitor and a suggested baseline. Combine the `xgb_predictor` and the provided `my_monitor` to configure the monitoring schedule for _hourly_ monitoring.

In [17]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

In [18]:
my_monitor.suggest_baseline(
    baseline_dataset=f's3://{bucket}/data/train.csv',
    dataset_format=DatasetFormat.csv(header=False),
)


Job Name:  baseline-suggestion-job-2022-01-14-12-53-12-764
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-934110659707/data/train.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-934110659707/model-monitor/baselining/baseline-suggestion-job-2022-01-14-12-53-12-764/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
.............................2022-01-14 12:57:46,014 - matplotlib.font_manager - INFO - Generating new fontManager, this may take some time...
2022-01-14 12:57:46.572960: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cann

Below, provide the monitoring schedule:

In [21]:
# TODO
my_monitor.delete_monitoring_schedule()
from sagemaker.model_monitor import CronExpressionGenerator

my_monitor.create_monitoring_schedule(
    monitor_schedule_name='wine-monitoring-schedule',
    endpoint_input=xgb_predictor.endpoint_name,
    statistics=my_monitor.baseline_statistics(),
    constraints=my_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)


Deleting Monitoring Schedule with name: wine-monitoring-schedule


Great job! You can check that your schedule was created by selecting the `SageMaker components and registries` tab on the far left.

In this exercise you configured Model Monitor to watch a simple model. Next, we'll monitor the same deployment for explainability.

__REMINDER:__ Don't leave your model deployed overnight. If you aren't going to follow up with the Clarify exercise within a few hours, use the code below to remove your model:

In [22]:
monitors = xgb_predictor.list_monitors()
for monitor in monitors:
    monitor.delete_monitoring_schedule()


Deleting Monitoring Schedule with name: wine-monitoring-schedule


In [23]:
xgb_predictor.delete_endpoint()

## Clarify

For the last exercise we'll deploy an explainability monitor using Clarify. We're going to use the model that you deployed in the last exercise, but if you cleaned up your deployments from the previous exercise, that's ok! You can rerun the deployment from the previous exercise up to the point where we deployed our model. It'll look like this:

```python
xgb_predictor = model.deploy(
    initial_instance_count=1, instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)
```

Once your model is deployed, you can come back here. _REMINDER_: you need to clean up your deployment, don't leave it running overnight. We'll provide some code at the end to delete your deployment.

## Prep

We'll begin by reloading our data from the previous exercise.

In [71]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

We also need to put the target variable in the first column per the docs for our chosen algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [72]:
df["TARGET"] = data['target']
df.set_index(df.pop('TARGET'), inplace=True)
df.reset_index(inplace=True)

Now we'll upload the data to S3 as train and validation data:

In [73]:
delimiter = int(len(df)/2)
train, test = df.iloc[delimiter:], df.iloc[:delimiter]

train.to_csv("train.csv", header=False, index=False)
test.to_csv("validation.csv", header=False, index=False)

val_location = session.upload_data('./validation.csv', key_prefix="data")
train_location = session.upload_data('./train.csv', key_prefix="data")

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

Great! Our data is staged and our model is deployed - let's monitor it for explainability. We need to define three config objects, the `SHAPConfig`, the `ModelConfig`, and the `ExplainabilityAnalysisConfig`. Below, we provide the `SHAPConfig`.

In [74]:
shap_config = sagemaker.clarify.SHAPConfig(
    baseline=[train.mean().astype(int).to_list()[1:]],
    num_samples=int(train.size),
    agg_method="mean_abs",
    save_local_shap_values=False,
)

Next up, fill in the blanks to define the `ModelConfig` and `ExplainabilityAnalysisConfig`.

In [75]:
# TODO
model_config = sagemaker.clarify.ModelConfig(
    model_name=xgb_predictor.endpoint_name,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    content_type="text/csv",
    accept_type="text/csv",
)

analysis_config = sagemaker.model_monitor.ExplainabilityAnalysisConfig(
        explainability_config=shap_config,
        model_config=model_config,
        headers=train.columns.to_list()[1:],
    )

Before we apply our config, we need to create the monitor object. This is what we'll apply all our config to.

In [76]:
model_explainability_monitor = sagemaker.model_monitor.ModelExplainabilityMonitor(
    role=role,
    sagemaker_session=session,
    max_runtime_in_seconds=1800,
)

Everything's ready! Below, create a monitoring schedule using the configs we created. Set the schedule to run _daily_.

In [77]:
# TODO 
from sagemaker.model_monitor import CronExpressionGenerator


explainability_uri = f"s3://{bucket}/model_explainability"
model_explainability_monitor.create_monitoring_schedule(
    output_s3_uri=explainability_uri,
    analysis_config=analysis_config,
    endpoint_input=xgb_predictor.endpoint_name,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)

Way to go! You can check that your schedule was created by selecting the `SageMaker components and registries` tab on the far left.

In this exercise you deployed a monitor for explainability to your SageMaker endpoint. This is the last exercise - you'll apply these learnings again in your Project at the end of the course.



__REMINDER:__ Don't leave your model deployed overnight. Use the code below to remove your model:

In [126]:
monitors = xgb_predictor.list_monitors()
for monitor in monitors:
    monitor.delete_monitoring_schedule()


Deleting Monitoring Schedule with name: monitoring-schedule-2021-09-13-17-25-08-560

Deleting Monitoring Schedule with name: wine-monitoring-schedule


In [127]:
xgb_predictor.delete_endpoint()